# Dogs vs Cats - Kaggle competition
https://www.kaggle.com/competitions/dogs-vs-cats/overview


1. **Problem definition**
In this competition, you'll write an algorithm to classify whether images contain either a dog or a cat.  This is easy for humans, dogs, and cats. Your computer will find it a bit more difficult.

2. **Data**
Web services are often protected with a challenge that's supposed to be easy for people to solve, but difficult for computers. Such a challenge is often called a CAPTCHA (Completely Automated Public Turing test to tell Computers and Humans Apart) or HIP (Human Interactive Proof). HIPs are used for many purposes, such as to reduce email and blog spam and prevent brute-force attacks on web site passwords. <br>
Asirra (Animal Species Image Recognition for Restricting Access) is a HIP that works by asking users to identify photographs of cats and dogs. This task is difficult for computers, but studies have shown that people can accomplish it quickly and accurately. Many even think it's fun! Here is an example of the Asirra interface:<br>
Asirra is unique because of its partnership with Petfinder.com, the world's largest site devoted to finding homes for homeless pets. They've provided Microsoft Research with over three million images of cats and dogs, manually classified by people at thousands of animal shelters across the United States. Kaggle is fortunate to offer a subset of this data for fun and research. 

    **The training archive contains 25,000 images of dogs and cats. <br>
    Train your algorithm on these files and predict the labels for test1.zip (1 = dog, 0 = cat).**
3. **Evaluation**<br>
Performance is evaluated on the percentage of correctly labeled images. (**Accuracy**)
4. **Submission**
Your submission should have a header. For each image in the test set, predict a label for its id (1 = dog, 0 = cat):


In [23]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import layers, Model
from shutil import copyfile
import datetime

# import the InceptionV3 network (not used)
from tensorflow.keras.applications.inception_v3 import InceptionV3

**Working with Tensorflow version 2.8.0**

# Download data

In [ ]:
# LARGE dataset
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Organize folders/sub-folders

In [4]:
source_path = '/tmp/PetImages'
source_path_dogs = os.path.join(source_path, 'Dog')
source_path_cats = os.path.join(source_path, 'Cat')

print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")

There are 12501 images of dogs.
There are 12501 images of cats.


In [6]:
root_dir = '/tmp/cats-v-dogs'
train_dir = os.path.join(root_dir, 'training')
test_dir = os.path.join(root_dir, 'testing')

# create primary directories to use with ImageDataGenerator
os.makedirs(root_dir)
os.makedirs(train_dir)
os.makedirs(test_dir)

# create subdirectories for train/test
os.makedirs(os.path.join(train_dir, 'cats'))
os.makedirs(os.path.join(train_dir, 'dogs'))
os.makedirs(os.path.join(test_dir, 'cats'))
os.makedirs(os.path.join(test_dir, 'dogs'))

In [9]:
# define directory variables
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"

TRAINING_DIR = "/tmp/cats-v-dogs/training/"
TESTING_DIR = "/tmp/cats-v-dogs/testing/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
TESTING_CATS_DIR = os.path.join(TESTING_DIR, "cats/")
TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
TESTING_DOGS_DIR = os.path.join(TESTING_DIR, "dogs/")

In [8]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  # get all images - either dogs or cats
  all_imgs = os.listdir(SOURCE)
  all_imgs_good = []

  # filter empty images = size 0 files
  for img in all_imgs:
    if os.path.getsize(os.path.join(SOURCE, img)) != 0:
      all_imgs_good.append(img)
    else:
      print(f"{img} is zero length, so ignoring.")

  # shuffle randomly all the non-empty images
  all_imgs_good = random.sample(all_imgs_good, len(all_imgs_good)) 

  # first part train, last part test
  train_list = all_imgs_good[0:int(SPLIT_SIZE * len(all_imgs_good))]
  test_list = all_imgs_good[int(SPLIT_SIZE * len(all_imgs_good)):]

  # save each image in the proper folder
  for img in train_list:
    copyfile(os.path.join(SOURCE, img), os.path.join(TRAINING, img))
  for img in test_list:
    copyfile(os.path.join(SOURCE, img), os.path.join(TESTING, img))


In [ ]:
# Define proportion of images used for training
split_size = .9

# Run the function
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Check that the number of images matches the expected output
print(f"\n\nThere are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(TESTING_CATS_DIR))} images of cats for testing")
print(f"There are {len(os.listdir(TESTING_DOGS_DIR))} images of dogs for testing")

# Create ImageDataGenerators

In [11]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  # use the ImageDataGenerator to stream the images to model
  # we can also use this to further augment the data to decrease overfitting
  # the values are typical values found in github projects


  # ==== TRAIN ==== #
  train_datagen = ImageDataGenerator(rescale=1.0 / 255.,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
  
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  
  # ==== TEST ==== #
  # no augmentation
  validation_datagen = ImageDataGenerator(rescale=1.0 / 255.)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                            batch_size=20,
                                                            class_mode='binary',
                                                            target_size=(150, 150))
  return train_generator, validation_generator


In [ ]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)

# Create model

In [ ]:
# # Transfer Learning - using a pre-trained Inception V3 network
# # Download the weights
# !wget --no-check-certificate \
#     https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
#     -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# # Create an instance of the inception model from the local pre-trained weights
# local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'   

# # initialize with no weight and then set the weights to the downloaded ones
# pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
#                                   include_top = False, 
#                                   weights = None) 
# pre_trained_model.load_weights(local_weights_file)

# # make all the layers in the pre-trained model non-trainable
# for layer in pre_trained_model.layers:
#   layer.trainable = False

# # extract only a part of the network
# last_desired_layer = pre_trained_model.get_layer('mixed7')
# last_output = last_desired_layer.output

# # create the big model using the functional API
# x = layers.Flatten()(last_output)
# x = layers.Dense(1024, activation='relu')(x)
# x = layers.Dropout(0.2)(x)  
# x = layers.Dense(1, activation='sigmoid')(x)        
# model = Model(inputs=pre_trained_model.input, outputs=x)

# # Compile the model
# model.compile(optimizer = RMSprop(learning_rate=0.0001), 
#               loss = 'binary_crossentropy',
#               metrics = ['accuracy'])

In [24]:
model = tf.keras.models.Sequential([  
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
model.compile(optimizer=RMSprop(learning_rate=0.0005), # or Adam, but its a bit slower
            loss='binary_crossentropy',
            metrics=['accuracy']) 

## Callbacks

In [26]:
# Early stopping function to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                  patience=3) #stops improving for 3 epochs


# Tensorboard - for vizualization
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("/tmp/logs",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

tensorboard = create_tensorboard_callback()

## Train

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=15,
                    verbose=1,
                    callbacks=[early_stopping, tensorboard])

# Vizualize

In [32]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /tmp/logs

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,6))
plt.title('Training and validation accuracy')
ax[0].plot(epochs, acc, 'r', "Training Accuracy")
ax[0].plot(epochs, val_acc, 'b', "Validation Accuracy")

ax[1].plot(epochs, loss, 'r', "Training Loss")
ax[1].plot(epochs, val_loss, 'b', "Validation Loss")

plt.show()